In [1]:
import numbers
import copy
import time
import numpy as np
import networkx as nx

from gtn.dataloader.graphcollection import GraphCollection
from gtn.dataloader.preprocessing import networkx_to_sparsegraph

from graph_gen import initial_attractiveness
from ged import calc_all_geds, calc_ged

In [2]:
rnd_state = np.random.RandomState(2)

## Generate graphs

In [74]:
# 20 clusters: 12 train, 4 val, 4 test
ngraphs = 20
min_nodes = 10000
total_nodes = np.arange(1 * min_nodes, 3 * min_nodes, dtype=int)
initial_nodes = np.arange(1, 5, dtype=int)
initial_attr = np.arange(5, dtype=int)

nx_graphs = []
for _ in range(ngraphs):
    n = rnd_state.choice(total_nodes)
    m = rnd_state.choice(initial_nodes)
    A = rnd_state.choice(initial_attr)
    nx_graphs.append(initial_attractiveness(n, m, A, rnd_state=rnd_state))

In [75]:
ns = [graph.number_of_nodes() for graph in nx_graphs]
es = [graph.number_of_edges() for graph in nx_graphs]

print(f"Train: mean nodes: {np.mean(ns[:12]):.2f}, edges: {np.mean(es[:12]):.2f}")
print(f"Val:   mean nodes: {np.mean(ns[12:16]):.2f}, edges: {np.mean(es[12:16]):.2f}")
print(f"Test:  mean nodes: {np.mean(ns[16:]):.2f}, edges: {np.mean(es[16:]):.2f}")

Train: mean nodes: 21781.25, edges: 41542.42
Val:   mean nodes: 19925.50, edges: 60196.75
Test:  mean nodes: 18257.50, edges: 39482.50


### Label nodes and edges

In [76]:
def label_graph(graph, num_nodelabels, num_edgelabels):
    nodelabels = rnd_state.randint(num_nodelabels, size=len(graph.nodes))
    for i, label in enumerate(nodelabels):
        graph.nodes[i]['label'] = label
    
    if num_edgelabels is not None:
        edgelabels = rnd_state.randint(num_edgelabels, size=len(graph.edges))
        for i, (n1, n2) in enumerate(graph.edges):
            graph.edges[n1, n2]['label'] = edgelabels[i]
        
num_nodelabels = 6
num_edgelabels = None #4

for graph in nx_graphs:
    label_graph(graph, num_nodelabels, num_edgelabels)

In [77]:
train_nx = nx_graphs[:12]
val_nx = nx_graphs[12:16]
test_nx = nx_graphs[16:]

### Edit graphs to generate trees (clusters)

In [78]:
def edit_graph(graph, num_nodelabels, num_edgelabels):
    edit_types = ['node_edit', 'node_add']
    edit_probs = np.array([4, 1], dtype=float)
    if graph.number_of_nodes() > 2:
        edit_types += ['node_del']
        edit_probs = np.append(edit_probs, 1)
    max_nedges = graph.number_of_nodes() * (graph.number_of_nodes() - 1) / 2
    if (graph.number_of_nodes() > 1
            and graph.number_of_edges() < max_nedges):
        edit_types += ['edge_add']
        edit_probs = np.append(edit_probs, 3)
    if graph.number_of_edges() > 0 and num_edgelabels is not None:
        edit_types += ['edge_edit']
        edit_probs = np.append(edit_probs, 4)
    if graph.number_of_edges() > 1:
        edit_types += ['edge_del']
        edit_probs = np.append(edit_probs, 3)
    edit_probs /= edit_probs.sum()
    
    edit_type = rnd_state.choice(edit_types, p=edit_probs)
    
    if edit_type == 'node_edit':
        inode = rnd_state.choice(graph.nodes)
        graph.nodes[inode]['label'] = rnd_state.randint(num_nodelabels)
        
    elif edit_type == 'node_add':
        # Add a node with 2 edges
        iadd = [inode for inode in range(graph.number_of_nodes() + 1) if inode not in graph.nodes][0]
        connect_probs = np.array(list(dict(graph.degree).values()), dtype=float)  # Corresponds to Barabasi-Albert
        connect_probs /= connect_probs.sum()
        iconnects = rnd_state.choice(graph.nodes, p=connect_probs, size=2, replace=False)
        iadds = np.repeat(iadd, 2)
        graph.add_edges_from(zip(iadds, iconnects))
        if num_edgelabels is not None:
            for pair_idx in zip(*(iadds, iconnects)):
                graph.edges[pair_idx]['label'] = rnd_state.randint(num_edgelabels)
        graph.nodes[iadd]['label'] = rnd_state.randint(num_nodelabels)
        
    elif edit_type == 'node_del':
        del_probs = 1 / (np.array(list(dict(graph.degree).values())) + 1)  # Inverse Initial Attractiveness with A=1
        del_probs /= del_probs.sum()
        inode = rnd_state.choice(graph.nodes, p=del_probs)
        graph.remove_node(inode)
        
    elif edit_type == 'edge_edit':
        iedge = rnd_state.choice(len(graph.edges))
        edge_idx = list(graph.edges)[iedge]
        graph.edges[edge_idx]['label'] = rnd_state.randint(num_edgelabels)
        
    elif edit_type == 'edge_add':
        connect_probs = np.array(list(dict(graph.degree).values()), dtype=float)  # Corresponds to Barabasi-Albert
        connect_probs /= connect_probs.sum()
        iconnects = rnd_state.choice(graph.nodes, p=connect_probs, size=2, replace=False)
        while graph.has_edge(*iconnects):
            iconnects = rnd_state.choice(graph.nodes, p=connect_probs, size=2, replace=False)
        graph.add_edge(*iconnects)
        if num_edgelabels is not None:
            graph.edges[iconnects]['label'] = rnd_state.randint(num_edgelabels)
        
    elif edit_type == 'edge_del':
        iedge = rnd_state.choice(len(graph.edges))
        edge_idx = list(graph.edges)[iedge]
        graph.remove_edge(*edge_idx)
        
    else:
        raise ValueError(f"Unknown edit type: {edit_type}")


def gen_edited_graph(graphs, num_nodelabels, num_edgelabels, min_edits=2, max_edits=5):
    g = nx_graphs[rnd_state.randint(len(nx_graphs))].copy()
    nedits = rnd_state.randint(min_edits, max_edits + 1)
    for _ in range(nedits):
        edit_graph(g, num_nodelabels, num_edgelabels)
        
    # Sanitize node labels
    relabel_dict = dict(zip(g.nodes, np.arange(g.number_of_nodes())))
    g = nx.relabel_nodes(g, relabel_dict)
    
    return g
    

def add_graph_variants(graphs, nadd, num_nodelabels, num_edgelabels, min_edits, max_edits):
    for _ in range(nadd):
        graphs.append(gen_edited_graph(graphs, num_nodelabels, num_edgelabels, min_edits, max_edits))

min_edits = min_nodes // 20
max_edits = min_nodes // 10
add_graph_variants(train_nx, len(train_nx) * 11, num_nodelabels, num_edgelabels, min_edits, max_edits)
add_graph_variants(val_nx, len(val_nx) * 11, num_nodelabels, num_edgelabels, min_edits, max_edits)
add_graph_variants(test_nx, len(test_nx) * 11, num_nodelabels, num_edgelabels, min_edits, max_edits)

In [79]:
def nnodes_nedges(nx_graphs):
    ns = [graph.number_of_nodes() for graph in nx_graphs]
    es = [graph.number_of_edges() for graph in nx_graphs]
    return ns, es

print(f"Dataset lengths: {[len(dataset) for dataset in [train_nx, val_nx, test_nx]]}")
print(f"Train edges: {nnodes_nedges(train_nx)[1]}")
print(f"Train: mean nodes, edges: {[np.mean(ns) for ns in nnodes_nedges(train_nx)]}")
print(f"Val:   mean nodes: {[np.mean(ns) for ns in nnodes_nedges(val_nx)]}")
print(f"Test:  mean nodes: {[np.mean(ns) for ns in nnodes_nedges(test_nx)]}")

Dataset lengths: [144, 48, 48]
Train edges: [10529, 28355, 75963, 12792, 59805, 23796, 79358, 44318, 90131, 28885, 22105, 22472, 75802, 74193, 74232, 35669, 28410, 35599, 22116, 54933, 22536, 22202, 33406, 57617, 57411, 12846, 75813, 20356, 59738, 44159, 22207, 59742, 28964, 33475, 46502, 74953, 74282, 54976, 10635, 35745, 12788, 12756, 74874, 10586, 20372, 10545, 28932, 59776, 22484, 46683, 74865, 89796, 22151, 20389, 54939, 79111, 33487, 46605, 54921, 22126, 12863, 54980, 59747, 20467, 75003, 35701, 54988, 12801, 23795, 75813, 20423, 35675, 28412, 46600, 89925, 75823, 89890, 54948, 28344, 22149, 28859, 44151, 74089, 44194, 12795, 59750, 33419, 22480, 20366, 46588, 44194, 74185, 28304, 89892, 79278, 28920, 10545, 74961, 23813, 20414, 12812, 44147, 44179, 79021, 10651, 54955, 54958, 74945, 44191, 22435, 20442, 78986, 46597, 33436, 28890, 75768, 12832, 54966, 28416, 89796, 22537, 79096, 74923, 57499, 28428, 46663, 33407, 23819, 28944, 28932, 12839, 89895, 59825, 28381, 74201, 46600, 283

### Convert to SparseGraphs

In [80]:
def nx_to_gcoll(nx_graphs):
    gcoll = GraphCollection()
    for nx_graph in nx_graphs:
        sp_graph = networkx_to_sparsegraph(nx_graph, sparse_node_attrs=False, sparse_edge_attrs=False)
        gcoll.append(sp_graph)
    return gcoll

gcoll_train = nx_to_gcoll(train_nx)
gcoll_val = nx_to_gcoll(val_nx)
gcoll_test = nx_to_gcoll(test_nx)

### Save as GraphCollection

In [81]:
gust.io.save_to_npz(f"pref_att_{min_nodes}_train.npz", gcoll_train)
gust.io.save_to_npz(f"pref_att_{min_nodes}_val.npz", gcoll_val)
gust.io.save_to_npz(f"pref_att_{min_nodes}_test.npz", gcoll_test)

In [73]:
# Check consistency of NetworkX conversion and npz-loading/saving
gcoll2 = gust.io.load_from_npz(f"pref_att_{min_nodes}_train.npz")
for i, sp_graph in enumerate(gcoll2):
    nx_graph2 = gust.sparsegraph_to_networkx(sp_graph)
    if not (nx_graph2.nodes.data() == train_nx[i].nodes.data()):
        print("Node error")
    if num_edgelabels is None:
        for j, (u, v, data2) in enumerate(nx_graph2.edges.data()):
            # Weight entry in gcoll2, but not in train_nx. Problem?
            if not (u, v) in train_nx[i].edges:
                print("Edge error")
    else:
        for j, (u, v, data2) in enumerate(nx_graph2.edges.data()):
            if not (int(data2['label']) == train_nx[i].edges[u, v]['label']):
                print("Edge error")

## Calculate graph edit distances

Set costs for all edit operations

In [16]:
def cost_node_subst(node_i, node_j):
    return 0 if node_i == node_j else 1
def cost_edge_subst(edge_i, edge_j):
    return 0 if edge_i == edge_j else 1
costs = {'node_subst': cost_node_subst, 'node_del': 2, 'node_ins': 2,
         'edge_subst': cost_edge_subst, 'edge_del': 2, 'edge_ins': 2}

We calculate the graph edit distance by solving the following binary linear program:
\begin{align*}
    \underset{\mathbf{x}, \mathbf{y}}{\text{min}} \quad& \left[ \sum_{i \in V_1} \sum_{k \in V_2} \left( c_{i,k} - c_{i,\epsilon} - c_{\epsilon,k} \right) x_{i,k} + \sum_{ij \in E_1} \sum_{kl \in E_2} \left( c_{ij,kl} - c_{ij,\epsilon} - c_{\epsilon,kl} \right) y_{ij,kl} \right] + \gamma\\
    \text{with} \quad& \gamma = \sum_{i \in V_1} c_{i,\epsilon} + \sum_{k \in V_2} c_{\epsilon,k} + \sum_{ij \in E_1} c_{ij,\epsilon} + \sum_{kl \in E_2} c_{\epsilon,kl}\\
    \text{subject to} \quad& \sum_{k \in V_2} x_{i,k} \le 1 \quad\quad \forall i \in V_1\\
    & \sum_{i \in V_1} x_{i,k} \le 1 \quad\quad \forall k \in V_2\\
    & \sum_{kl \in E_2} y_{ij,kl} - x_{i,k} \le 0 \quad\quad \forall k \in V_2 ,\; \forall ij \in E_1\\
    & \sum_{kl \in E_2} y_{ij,kl} - x_{j,l} \le 0 \quad\quad \forall l \in V_2 ,\; \forall ij \in E_1\\
    \text{with} \quad& x_{i,k} \in \{0,1\} \quad\quad \forall (i,k) \in V_1 \times V_2\\
    & y_{ij,kl} \in \{0,1\} \quad\quad \forall (ij,kl) \in E_1 \times E_2
\end{align*}

In [83]:
import cvxopt
from cvxopt.glpk import ilp

def calc_ged_prototype(graph1, graph2, costs, timelimit=None):
    """
    Calculate the graph edit distance between two NetworkX graphs.
    
    Parameters
    ----------
    graph1 : NetworkX graph
        Graph, where all nodes and features have a 'label' attribute, which is an integer.
        
    graph2 : NetworkX graph
        Graph, where all nodes and features have a 'label' attribute, which is an integer.
        
    costs: dict
        Dictionary containing entries for all edit costs:
        'node_subst': Function: [Number, Number] -> Number
        'node_del': float
        'node_ins': float
        'edge_subst': Function: [Number, Number] -> Number
        'edge_del': float
        'edge_ins': float
        
    timelimit: int
        Timelimit of the glpk solver in seconds

    Returns
    -------
    dist : float
        Graph edit distance between the graphs
        
    nodes_perm : list(integer tuples)
        Permutation of nodes from graph1 to graph2
        
    edges_perm : list(tuples of integer tuples)
        Permutation of edges from graph1 to graph2
    """
    
    # Create costs vector c
    costs_nodes = np.zeros([graph1.number_of_nodes(), graph2.number_of_nodes()])
    costs_nodes -= costs['node_del'] + costs['node_ins']
    for i, data_i in graph1.nodes().data():
        for k, data_k in graph2.nodes().data():
            costs_nodes[i, k] += costs['node_subst'](data_i['label'], data_k['label'])
    
    costs_edges = np.zeros([graph1.number_of_edges(), graph2.number_of_edges()])
    costs_edges -= costs['edge_del'] + costs['edge_ins']
    for id1, (i, j, data_ij) in enumerate(graph1.edges().data()):
        for id2, (k, l, data_kl) in enumerate(graph2.edges().data()):
            costs_edges[id1, id2] += costs['edge_subst'](data_ij['label'], data_kl['label'])
    
    c = cvxopt.matrix(np.concatenate([costs_nodes.flatten(), costs_edges.flatten()]))
    x_size = costs_nodes.size + costs_edges.size
    
    # Create constraints vector h
    h_nodes = np.ones(graph1.number_of_nodes() + graph2.number_of_nodes())
    h_edges = np.zeros(2 * graph2.number_of_nodes() * graph1.number_of_edges())
    h = cvxopt.matrix(np.concatenate([h_nodes, h_edges]))
    
    # Create constraints matrix G
    G = np.zeros([h_nodes.size + h_edges.size, x_size])
    row = -1
    for i in graph1.nodes():
        row += 1
        for k in graph2.nodes():
            G[row, np.ravel_multi_index([i, k], costs_nodes.shape)] = 1
    for k in graph2.nodes():
        row += 1
        for i in graph1.nodes():
            G[row, np.ravel_multi_index([i, k], costs_nodes.shape)] = 1
    for k in graph2.nodes():
        for id1, (i, j) in enumerate(graph1.edges()):
            row += 1
            G[row, np.ravel_multi_index([i, k], costs_nodes.shape)] = -1
            for id2, (k_edge, l) in enumerate(graph2.edges()):
                if k == k_edge:
                    G[row, costs_nodes.size + np.ravel_multi_index([id1, id2], costs_edges.shape)] = 1
    for l in graph2.nodes():
        for id1, (i, j) in enumerate(graph1.edges()):
            row += 1
            G[row, np.ravel_multi_index([j, l], costs_nodes.shape)] = -1
            for id2, (k, l_edge) in enumerate(graph2.edges()):
                if l == l_edge:
                    G[row, costs_nodes.size + np.ravel_multi_index([id1, id2], costs_edges.shape)] = 1
    G_cvxopt = cvxopt.matrix(G)
    
    # Create indices for integer/binary variables (everything's binary)
    I = set()
    B = set(range(x_size))
    
    # Solve binary linear problem
    if timelimit is None:
        options = None
    else:
        options = {'tm_lim': timelimit * 1000}
    status, x = ilp(c, G_cvxopt, h, None, None, I, B, options=options)
    # print(status)
    
    # Transform node indices
    x_ik = np.reshape(x[:costs_nodes.size], costs_nodes.shape)
    assert np.all((0 <= x_ik.sum(axis=0)) & (x_ik.sum(axis=0) <= 1)), "Node assignment column sum outside allowed range (0, 1)"
    assert np.all((0 <= x_ik.sum(axis=1)) & (x_ik.sum(axis=1) <= 1)), "Node assignment row sum outside allowed range (0, 1)"
    nodes_transf = [None, None]
    nodes_transf[0], nodes_transf[1] = np.where(x_ik > 0)
    nodes_transf[0] = list(nodes_transf[0])
    nodes_transf[1] = list(nodes_transf[1])
    
    # Transform edge indices
    y_id1id2 = np.reshape(x[costs_nodes.size:], costs_edges.shape)
    assert np.all((0 <= y_id1id2.sum(axis=0)) & (y_id1id2.sum(axis=0) <= 1)), "Edge assignment column sum outside allowed range (0, 1)"
    assert np.all((0 <= y_id1id2.sum(axis=1)) & (y_id1id2.sum(axis=1) <= 1)), "Edge assignment row sum outside allowed range (0, 1)"
    edges_id_perm = zip(*np.where(y_id1id2 > 0))
    edges_transf = [[], []]
    edges1 = list(graph1.edges())
    edges2 = list(graph2.edges())
    for id1, id2 in edges_id_perm:
        edges_transf[0].append(edges1[id1])
        edges_transf[1].append(edges2[id2])
    
    # Calculate graph edit distance
    dist = ((x_ik * costs_nodes).sum() + (y_id1id2 * costs_edges).sum()
            + graph1.number_of_nodes() * costs['node_del'] + graph2.number_of_nodes() * costs['node_ins']
            + graph1.number_of_edges() * costs['edge_del'] + graph2.number_of_edges() * costs['edge_ins'])
    
    # Add entries for insertions and deletions
    for node in graph1.nodes():
        if node not in nodes_transf[0]:
            nodes_transf[0].append(node)
            nodes_transf[1].append(np.nan)
    for node in graph2.nodes():
        if node not in nodes_transf[1]:
            nodes_transf[0].append(np.nan)
            nodes_transf[1].append(node)
    for edge in graph1.edges():
        if edge not in edges_transf[0]:
            edges_transf[0].append(edge)
            edges_transf[1].append(np.nan)
    for edge in graph2.edges():
        if edge not in edges_transf[1]:
            edges_transf[0].append(np.nan)
            edges_transf[1].append(edge)
    
    # Transform permutation list
    nodes_perm = list(zip(*nodes_transf))
    edges_perm = list(zip(*edges_transf))
        
    return dist, nodes_perm, edges_perm

In [84]:
calc_all_geds(gcoll, costs, nprocesses=4, timelimit=1000)
print(gcoll.dists.A)

[[ 0. 35.]
 [35.  0.]]


/nfs/homedirs/klicpera/anaconda3/envs/graph/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [85]:
g1 = gust.sparsegraph_to_networkx(gcoll[0])
g2 = gust.sparsegraph_to_networkx(gcoll[1])
dist, nodes_perm, edges_perm = calc_ged_prototype(g1, g2, costs)